#  GENERATE INPUTS

In [1]:
tbitmap = {
    '0000' : 'f',    '0001' : 'g',    '0010' : 'h',    '0011' : 'i',
    '0100' : 'j',    '0101' : 'k',    '0110' : 'l',    '0111' : 'm',
    '1000' : 'n',    '1001' : 'o',    '1010' : 'p',    '1011' : 'q',
    '1100' : 'r',    '1101' : 's',    '1110' : 't',    '1111' : 'u'
}

tcharmap = {
    'f' : '0000' ,    'g' : '0001' ,    'h' : '0010' ,    'i' : '0011' ,
    'j' : '0100' ,    'k' : '0101' ,    'l' : '0110' ,    'm' : '0111' ,
    'n' : '1000' ,    'o' : '1001' ,    'p' : '1010' ,    'q' : '1011' ,
    'r' : '1100' ,    's' : '1101' ,    't' : '1110' ,    'u' : '1111' 
}

In [2]:
def gen_inputs(loc, clean = 1 ):
    file = open(loc,"w")
    if( clean == 0 ):
        file.write('Kerberos\n')
        file.write('Kerberos@2021\n')
        file.write('5\n')
        file.write('go\n')
        file.write('wave\n')
        file.write('dive\n')
        file.write('go\n')
        file.write('read\n')
        file.write('password\n')
        file.write('c\n')
        
    for i in range(8):
        for j in range(128):
            binary_block = bin(j)[2:].zfill(8)
            outs = ''
            for k in range(8):
                if( k==i ):
                    outs = outs + tbitmap[binary_block[:4]] + tbitmap[binary_block[4:]]
                else:
                    outs = outs + 'ff'
            file.write(outs)
            if( clean == 0 ):
                file.write("\nc\n")
            else:
                file.write(' ')
        if( clean == 1 ):
            file.write('\n')
    if( clean == 0 ):
        file.write('back\n')
        file.write('exit\n')
    file.close()

In [3]:
gen_inputs('input_clean.txt', 1)

# CLEAN OUTPUT

In [4]:
def clean_output():
    ct1 = []

    file1 = open("output.txt","r")
    for i in range(139):
        file1.readline()

    for t in file1:
        ct1.append(t[2:18])
        for i in range(14):
            file1.readline()

    ciphertexts = ct1[:-1]

    file = open('output_clean.txt',"w")
    for i in range(8):
        for j in range(128):
            file.write(ciphertexts[i*128 + j])
            file.write(' ')
        file.write('\n')
        

In [5]:
clean_output()

# BREAK AES

In [6]:
# BYTES CHAR INT CONVERSTIONS
def int2blocks( inbyte ):
    inbyte = bin(inbyte)[2:].zfill(8)
    return tbitmap[inbyte[0:4]] + tbitmap[inbyte[4:8]]

def block2ints( block ):
    byte = []
    for i in range(0, len(block), 2):
        byte.append( int( tcharmap[block[i]] + tcharmap[block[i+1]], 2) )
    return byte

In [7]:
# AES FUNCTIONS
from pyfinite import ffield

F = ffield.FField(7)
exp_temp = [[-1]*128 for i in range(128)]

def Xor(a, b):
    return int(a) ^ int(b)

def M(a, b):                  #Field Multiplication
    return F.Multiply(a, b)

def E(num, power):            # Exponentiation
    if exp_temp[num][power] != -1:
        return exp_temp[num][power]

    result = 0
    if power == 0:
        result = 1
    elif power == 1:
        result = num
    elif power%2 == 0:
        temp = E(num, power>>1)
        result = M(temp, temp)
    else:
        temp = E(num, power>>1)
        result = M(temp, temp)
        result = M(num, result)

    exp_temp[num][power] = result
    return result


def A(matrix, block):     #Affine Transformation
    result = [0]*8
    for row, element in zip(matrix, block):
        scalarMultiply = []
        for i in range(len(row)):
            scalarMultiply.append( M(row[i], element) )
        for i in range(len(result)):
            result[i] = Xor( result[i], scalarMultiply[i])  
    return result

def EAEAE(pt, AM, EM):
    outE = []
    for i in range( len(pt)):
        outE.append( E( pt[i], EM[i] ) )
    outA = A(AM, outE)
    outE = []
    for i in range( len(pt)):
        outE.append( E( outA[i], EM[i] ) )
    outA = A(AM, outE)
    outE = []
    for i in range( len(pt)):
        outE.append( E( outA[i], EM[i] ) )
    return outE

In [8]:
def read_files(loc):
    listtext = []
    file = open(loc, 'r')
    lines = file.readlines()
    for line in lines:
        pts = line.split(' ')
        listtext.append(pts[:-1])
    return listtext

In [9]:
pt = read_files('input_clean.txt')
ct = read_files('output_clean.txt')
for i in range(len(pt)):
    for j in range( len(pt[i])):
        pt[i][j] = block2ints(  pt[i][j] )
        ct[i][j] = block2ints(  ct[i][j] )   

In [10]:
# Finding possible diagonals of A matrix and E Matrix
E_temps = [[] for i in range(8)]
A_temps = [[[] for i in range(8)] for j in range(8)]
for index in range(8):
    pairs = []
    for i in range(len(pt)):
        pairs.append( (pt[index][i][index],ct[index][i][index]) )
    
    for i in range(1,127):
        for j in range(1,128):
            flag = True
            for x in pairs:
                if x[1] != E(M(E(M(E( x[0], i), j), i), j), i):
                    flag = False
                    break
            if flag:
                E_temps[index].append(i)
                A_temps[index][index].append(j) 

A_temps, E_temps

([[[27, 84, 84], [], [], [], [], [], [], []],
  [[], [72, 101, 70], [], [], [], [], [], []],
  [[], [], [43, 14, 72], [], [], [], [], []],
  [[], [], [], [38, 113, 12], [], [], [], []],
  [[], [], [], [], [118, 112, 20], [], [], []],
  [[], [], [], [], [], [11, 106, 70], [], []],
  [[], [], [], [], [], [], [123, 27, 63], []],
  [[], [], [], [], [], [], [], [38, 61, 125]]],
 [[1, 19, 107],
  [53, 83, 118],
  [40, 89, 125],
  [4, 47, 76],
  [40, 89, 125],
  [40, 89, 125],
  [18, 21, 88],
  [15, 31, 81]])

In [11]:
# Finding Exact possible diagonal elemnt of A and eleents of E matrix
for index in range(7):
    pairs = []
    for i in range(len(pt)):
        pairs.append( (pt[index][i][index],ct[index][i][index+1]) )
            
    for i in range(1,128):
        for ei, ai in zip(E_temps[index], A_temps[index][index]):
            for ei1, ai1 in zip(E_temps[index+1], A_temps[index+1][index+1]):
            
                flag = True
                for x in pairs:
                    if x[1] != E(Xor( M(E(M(E(x[0], ei), ai), ei), i) ,
                                      M(E(M(E(x[0], ei), i), ei1), ai1)) ,
                                     ei1):
                        flag = False
                        break
                if flag:
                    E_temps[index+1]          = [ei1]
                    E_temps[index]            = [ei]
                    A_temps[index+1][index+1] = [ai1]
                    A_temps[index][index]     = [ai]
                    A_temps[index][index+1]   = [i]
                
A_temps, E_temps

([[[84], [115], [], [], [], [], [], []],
  [[], [70], [28], [], [], [], [], []],
  [[], [], [43], [12], [], [], [], []],
  [[], [], [], [12], [119], [], [], []],
  [[], [], [], [], [112], [111], [], []],
  [[], [], [], [], [], [11], [93], []],
  [[], [], [], [], [], [], [27], [25]],
  [[], [], [], [], [], [], [], [38]]],
 [[19], [118], [40], [76], [89], [40], [21], [15]])

In [12]:
# Generating Exact A and E matrix
EM = []
AM = []
for i in range(8):
    EM.append( E_temps[i][0] )
    lst = []
    for j in range(8):
        if( A_temps[i][j] == [] ):
            lst.append(0)
        else:
            lst.append( A_temps[i][j][0])
    AM.append(lst)
    
AM,EM

([[84, 115, 0, 0, 0, 0, 0, 0],
  [0, 70, 28, 0, 0, 0, 0, 0],
  [0, 0, 43, 12, 0, 0, 0, 0],
  [0, 0, 0, 12, 119, 0, 0, 0],
  [0, 0, 0, 0, 112, 111, 0, 0],
  [0, 0, 0, 0, 0, 11, 93, 0],
  [0, 0, 0, 0, 0, 0, 27, 25],
  [0, 0, 0, 0, 0, 0, 0, 38]],
 [19, 118, 40, 76, 89, 40, 21, 15])

In [13]:
# Computing Remaining Elements

for diagindex in range(6):
    #print()
    #print(diagindex, end = '  : ')
    offset = diagindex + 2
    
    for index in range(len(pt)):
        if( index > 7 - offset):
            continue
        
        #print(index+offset, end = '  ,')
        pts = pt[index]
        cts = ct[index]
        
        for i in range(1,128):
            AM[index][index + offset] = i
            flag = True
            for j in range(len( pts )):
                if( EAEAE( pts[j], AM, EM ) [index + offset] != cts[j][index + offset] ):
                    flag = False
                    break
            if flag:
                A_temps[index][index + offset] = [i]
        AM[index][index+offset] = A_temps[index][index + offset][0]

print('Final Matrices')
AM,EM

Final Matrices


([[84, 115, 14, 97, 96, 19, 11, 76],
  [0, 70, 28, 21, 43, 53, 120, 12],
  [0, 0, 43, 12, 12, 18, 20, 95],
  [0, 0, 0, 12, 119, 41, 101, 22],
  [0, 0, 0, 0, 112, 111, 30, 21],
  [0, 0, 0, 0, 0, 11, 93, 72],
  [0, 0, 0, 0, 0, 0, 27, 25],
  [0, 0, 0, 0, 0, 0, 0, 38]],
 [19, 118, 40, 76, 89, 40, 21, 15])

# Decrypting the Password

In [14]:
encrypted_password = 'gsgqljjkfgglmufrifkfhjjtmikjflkr'

In [15]:
enc_ints = block2ints(  encrypted_password )

In [16]:
def DecryptPassword(password):
    output = ""
    for ind in range(8):
        for ans in range(128):
            inp = output + int2blocks(ans) + (14-len(output))*'f'
            if (password[ind]) == EAEAE( block2ints(inp), AM, EM)[ind]:
                output += int2blocks(ans)
                break
    return output

In [17]:
dec_password  = DecryptPassword(enc_ints[0:8])  +  DecryptPassword(enc_ints[8:16]) 
print(dec_password)
for i in block2ints(  dec_password ):
    print( chr(i) , end = '' )

mimmmgmhlglsltlrlgljifififififif
swqramnlad000000